In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [28]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z'/!?]+", r" ", s)
    s = re.sub(r"'", " '", s)
    return s.strip()

In [29]:
normalizeString("Yes sir. /uanaconda'close' ")

"yes sir /uanaconda 'close '"

In [30]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    lines = []
    counter = 0

    with open("/media/nathanmon/389E28739E282BB6/Users/Natha/Datasets/MyJarvisConversation/conversation.txt", "r") as f:
        for line in f.readlines():
            if line[0] == "U":
                lines.append("")
                lines[counter] += line[6:]
            elif line[0] == "J":
                lines[counter] += line[8:]
                counter += 1


    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\n')[:2]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [31]:
MAX_LENGTH = 20

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [32]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print("User: ", input_lang.n_words)
    print("Jarvis: ",output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('usr', 'jar', False)
print(random.choice(pairs))

Reading lines...
Read 241 sentence pairs
Trimmed to 241 sentence pairs
Counting words...
Counted words:
User:  286
Jarvis:  269
['thanks', 'sure']


In [33]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [34]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [35]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [36]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in normalizeString(sentence).split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'fra', False)
    train_pairs, val_pairs = pairs[:225], pairs[225:]

    train_n = len(train_pairs)
    val_n = len(val_pairs)
    train_input_ids = np.zeros((train_n, MAX_LENGTH), dtype=np.int32)
    train_target_ids = np.zeros((train_n, MAX_LENGTH), dtype=np.int32)
    val_input_ids = np.zeros((val_n, MAX_LENGTH), dtype=np.int32)
    val_target_ids = np.zeros((val_n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(train_pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        train_input_ids[idx, :len(inp_ids)] = inp_ids
        train_target_ids[idx, :len(tgt_ids)] = tgt_ids
        
    for idx, (inp, tgt) in enumerate(val_pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        val_input_ids[idx, :len(inp_ids)] = inp_ids
        val_target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(train_input_ids).to(device),
                               torch.LongTensor(train_target_ids).to(device))
    val_data = TensorDataset(torch.LongTensor(val_input_ids).to(device),
                               torch.LongTensor(val_target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    val_sampler = RandomSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)
    
    return input_lang, output_lang, train_dataloader, val_dataloader

In [37]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion, train=True):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        if train:
            loss.backward()

            encoder_optimizer.step()
            decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [38]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [39]:
def train(train_dataloader, val_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_train_losses = []
    print_train_loss_total = 0  # Reset every print_every
    plot_train_loss_total = 0  # Reset every plot_every
    
    print_val_loss_total = 0  # Reset every print_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        train_loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_train_loss_total += train_loss
        plot_train_loss_total += train_loss
        
        # Evaluate validation dataloader
        val_loss = train_epoch(val_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, train=False)
        print_val_loss_total += val_loss

        if epoch % print_every == 0:
            print_train_loss_avg = print_train_loss_total / print_every
            print_train_loss_total = 0
            print_val_loss_avg = print_val_loss_total / print_every
            print_val_loss_total = 0
            print('%s (%d %d%%) %.4f %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_train_loss_avg, print_val_loss_avg))

        if epoch % plot_every == 0:
            plot_train_loss_avg = plot_train_loss_total / plot_every
            plot_train_losses.append(plot_train_loss_avg)
            plot_train_loss_total = 0

    showPlot(plot_train_losses)

In [40]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [41]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [42]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [50]:
hidden_size = 512
batch_size = 16
dropout_p = 0.5

input_lang, output_lang, train_dataloader, val_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size, dropout_p=dropout_p).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=dropout_p).to(device)

train(train_dataloader, val_dataloader, encoder, decoder, 500, print_every=5, plot_every=5)

Reading lines...
Read 241 sentence pairs
Trimmed to 241 sentence pairs
Counting words...
Counted words:
User:  286
Jarvis:  269
0m 1s (- 2m 28s) (5 1%) 1.0180 0.6901
0m 2s (- 2m 24s) (10 2%) 0.4096 0.6341
0m 4s (- 2m 22s) (15 3%) 0.1995 0.6422
0m 5s (- 2m 20s) (20 4%) 0.1024 0.6630
0m 7s (- 2m 19s) (25 5%) 0.0544 0.6693
0m 8s (- 2m 17s) (30 6%) 0.0302 0.6849
0m 10s (- 2m 15s) (35 7%) 0.0155 0.6961
0m 11s (- 2m 14s) (40 8%) 0.0115 0.7030
0m 13s (- 2m 12s) (45 9%) 0.0102 0.7097
0m 14s (- 2m 9s) (50 10%) 0.0071 0.7243
0m 15s (- 2m 7s) (55 11%) 0.0056 0.7166
0m 17s (- 2m 5s) (60 12%) 0.0050 0.7363
0m 18s (- 2m 3s) (65 13%) 0.0045 0.7424
0m 19s (- 2m 1s) (70 14%) 0.0044 0.7351
0m 21s (- 1m 59s) (75 15%) 0.0041 0.7547
0m 22s (- 1m 57s) (80 16%) 0.0036 0.7619
0m 23s (- 1m 56s) (85 17%) 0.0037 0.7514
0m 25s (- 1m 54s) (90 18%) 0.0072 0.7610
0m 26s (- 1m 52s) (95 19%) 0.0041 0.7610
0m 27s (- 1m 51s) (100 20%) 0.0037 0.7755
0m 29s (- 1m 49s) (105 21%) 0.0036 0.7774
0m 30s (- 1m 48s) (110 22%) 0.

# Implement

In [44]:
ones_place = {"zero": 0, "one": 1, "two": 2, "three": 3, "four": 4, 
              "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9,
              "ten": 10, "eleven": 11, "twelve": 12, "thirteen": 13,
              "fourteen": 14, "fifteen": 15, "sixteen": 16, 
              "seventeen": 17, "eighteen": 18, "nineteen": 19}

tens_place = {"twenty": 20, "thirty": 30, "forty": 40, "fifty": 50,
              "sixty": 60, "seventy": 70, "eighty": 80, "ninety": 90}

hundreds_place = {"hundred": 10**2, "thousand": 10**3, "million": 10**6, "billion": 10**9,
                  "trillion": 10**12, "quadrillion": 10**15, "quintillion": 10**18}

# Convert word form to numbers
# Works for numbers up to the quintillions
# Also works for dates like nineteen sixty three of twenty twenty four
def word2num(word):
    digits = word.split()
    num = 0
    
    has_ones = False
    has_tens = False
    tens = 0
    hundreds = 0
    for i in range(len(digits)):
        digit = digits[i]
        
        if digit in ones_place:
            # In a real number, there would never be a ones place or tens place next to another ones place
            # For example, there is never nine nineteen or four ninety
            # However, this is seen in a year such as nineteen forty one
            if (has_tens and ones_place[digit] >= 10) or has_ones:
                num *= 100
                
            num += ones_place[digit]
            # Store the tens place and hundreds place in case there is a thousand or million in front of them
            tens += ones_place[digit]
            hundreds += ones_place[digit]
            # Store boolean values to see if there has been a tens place or ones place
            has_ones = True
            has_tens = False
    
        elif digit in tens_place:
            if has_ones or has_tens:
                # If someone enters a year(eighteen twelve works different than one thousand eight hundred twelve)
                num *= 100
                has_tens = False
            else:
                has_tens = True
            
            num += tens_place[digit]
            tens += tens_place[digit]
            hundreds += tens_place[digit]
            has_ones = False
            
        elif digit in hundreds_place:
            if digit != "hundred" and hundreds:
                num += hundreds * hundreds_place[digit] - hundreds
                hundreds = 0
                tens = 0
            else:
                num += tens * hundreds_place[digit] - tens
                hundreds = tens * hundreds_place[digit]
                tens = 0
            has_ones = False
            has_tens = False
        
    return num

In [45]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
}
def weather(city, wob):
    res = requests.get(
        f'https://www.google.com/search?q={city}+weather&rlz=1C1VDKB_enUS1016US1016&oq={city}&aqs=chrome.0.69i59j69i57j69i59l2j0i271l2j69i61l2.905j0j7&sourceid=chrome&ie=UTF-8',
        headers=headers
    )
    soup = bs4(res.text, 'html.parser')
    return soup.select(f"#wob_{wob}")[0].getText().strip()

def volume(vol):
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))

    volume.SetMasterVolumeLevelScalar(word2num(vol)/100, None)

In [49]:
while True:
    chat = input("Speak: ")
    output = evaluate(encoder, decoder, chat, input_lang, output_lang)
    response = ' '.join(output[0][:-1])
    
    print("Response: ", response)
    # Get the date
    if "/udate" in response:
        response = response.replace("/udate", datetime.date.today().strftime("%B %d, %Y"))
    # Get the time
    if "/utime" in response:
        response = response.replace("/utime", datetime.datetime.now().strftime("%I:%M:%S"))
    # Get the temperature
    if "/utemp" in response:
        response = response.replace("/utemp", weather("boston", "tm"))
    # Get the humidity
    if "/uhumidity" in response:
        response = response.replace("/uhumidity", weather("boston", "hm"))
    # Get the wind speed
    if "/uwind" in response:
        response = response.replace("/uwind", weather("boston", "ws"))
    # Get the amount of precipitation
    if "/uprecipitation" in response:
        response = response.replace("/uprecipitation", weather("boston", "pp"))
    if "/uvolume" in response:
        after = response.split("/uvolume")[-1]
        vol = after.split("'")[1]
        #response = response.replace("/uvolume"+"'"+vol+"'", "")
        volume(vol)
    if "/usleep" in response:
        response = response.replace("/usleep", "")
        print(response)
        break
    if "/unewtab" in response:
        response = response.replace("/unewtab", "")
        pyautogui.hotkey('ctrl', 't')
    if "/uclosetab" in response:
        response = response.replace("/uclosetab", "")
        pyautogui.hotkey('ctrl', 'w')
    if "/uswitchtab" in response:
        after = response.split("/uvolume")[-1]
        new = after.split("'")[1]
        response = response.replace("/uswitchtab"+"'"+new+"'", "")
        pyautogui.hotkey('ctrl', str(word2num(new)))
    
    print("Filtered: ", response)

Speak: volume sixteen
Response:  later graphical graphical /ufile out be be dinner out my teeth fifth dinner dinner dinner dinner dinner dinner dinner
Filtered:  later graphical graphical /ufile out be be dinner out my teeth fifth dinner dinner dinner dinner dinner dinner dinner
Speak: sleep
Response:  SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS
Filtered:  SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS
Speak: what's the date jarvis
Response:  SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS
Filtered:  SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS
Speak: what is the date
Response:  
Filtered:  
Speak: what is the date jarvis
Response:  
Filtered:  
Speak: what's the temperature
Response:  SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS
Filtered:  SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS


KeyboardInterrupt: Interrupted by user

In [ ]:
"f"

In [53]:
' '.join(response[:-1])

'goodbye sir usleep'